# Creating custom optimizers to use in NN

## Importing libraries

In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from ipynb.fs.full.Useful_funcs import data_pipeline, pre_model
from sklearn.datasets import fetch_california_housing

## Loading dataset

In [4]:
housing = fetch_california_housing()

In [5]:
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = data_pipeline(housing)

## Custom optimizer

In [17]:
class custom_optimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate = 0.001, momentum = 0.9, name = 'custom_optimizer', **kwargs):
        super().__init__(name, **kwargs)
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate)) # For storing hyperparameters and setting lr = learning_rate
        self._set_hyper('decay', self._initial_decay)
        self._set_hyper('momentum', momentum)
    def _create_slots(self, var_list):
        # For each model variable, create the optimizer variable associated with it. TF call these variables lots
        for var in var_list:
            self.add_slot(var, 'momentum') # For momentum optimization, we need a slot for each of the model variable.
    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # Handling learning rate decay
        momentum_var = self.get_slot(var, 'momentum')
        momentum_hyper = self._get_hyper('momentum', var_dtype)
        momentum_var.assign(momentum_var * momentum_hyper - (1. - momentum_hyper) * grad)
        var.assign_add(momentum_var * lr_t)
    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'learning_rate' : self._serialize_hyperparameter('learning_rate'), 'decay' : self._serialize_hyperparameter('decay'), 
                'momentum' : self._serialize_hyperparameter('momentum')}

In [18]:
pre_model()

In [19]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_shape = [8]))

In [20]:
model.compile(loss = 'mse', optimizer = custom_optimizer())

In [21]:
model.fit(x_train_scaled, y_train, epochs = 1)

363/363 [==============================] - 0s 571us/step - loss: 3.8128
